In [1]:
# Add erpsc to path
import os
import sys

In [2]:
import nltk
import itertools 

In [3]:
my_path = '/Users/vassiki/Desktop/nhw17/DataDrivenCognitiveOntology/'
#sys.path.append(os.getcwd)
sys.path.append(my_path)
# ^ Update the above with a link to the folder that has 'erpsc' in it, if not currently in path

In [4]:
# Import erpsc
from lisc.base import Base
from lisc.core.db import SCDB
from lisc.data import Data
from lisc.data_all import DataAll

In [7]:
# Add data to path
db = SCDB(False)
#fake_path = '/Users/vassiki/Desktop/nhw17/ERPSC_NLP/notebooks'
db.project_path = os.path.join(os.getcwd(), 'dat')
#db.project_path = fake_path
db.gen_paths()

In [8]:
# Load raw data from a particular ERP
dat = Data('p300')
dat.load(db)

In [9]:
def get_corpus(ds):
    """
    To flatten all abstract words into a long list 
    
    TO DO: Generalize to making a corpus of authors, or any metric the author wants
    
    Attributes
    ----------
    ds : Data structure returned by ERFSC when a query is submitted
    """
    aPreCorpus = []
    
    numArticles = ds.n_articles
    print('Iterating over %d articles for extracting words') %(ds.n_articles)
    for AIdx in xrange(numArticles):
        if ds.words[AIdx] != None:
            aPreCorpus.append(ds.words[AIdx])
        
    
    PreText = list(itertools.chain.from_iterable(aPreCorpus))
    
    Corpus = nltk.Text(PreText)
    
    return Corpus
        

In [10]:
def get_common_words(Corpus,NumWords):
    """
    Return the most common words in the concatenated article text corpus
    
    Attributes
    ----------
    Corpus: generated from keyword arguments from cognitive atlas
    NumWords: Top N words to return
    """
    
    assert(NumWords <= len(Corpus))
    
    fdist = nltk.FreqDist(Corpus)
    common_words = fdist.most_common(NumWords)
    header = [('Term', 'Frequency'),('-'*len('Term'),'-'*len('Frequency'))]
    wordList = header + common_words
    
    width = max(len(e) for t in wordList for e in t[:-1]) + 1 
    format=('%%-%ds' % width) * len(wordList[0])
    print '\n'.join(format % tuple(t) for t in wordList)
    print('\n Total Word Count : %d') %(len(Corpus))
    

In [18]:
def get_similarity(Corpus,ds,keyword='cognition'):
    """
    Returns the most similar words, determined by neighborhood in corpus
    
    Attributes
    ----------
    
    Corpus : concatenated list of abstracts
    ds : ouptut of lisc
    keyword : query
    
    NOTE: There is a bug in text.similarity in NLTK2.0, we want to use this function to return similar 
    words in a ranked fashion 
    """
    #similar_words = Corpus.similar(keyword)
    #return similar_words
    #similar_words = Corpus._word_contex_index.similar_words(keyword)
    #if similar_words != None:
     #   print '\n'.join(word for word in similar_words)
    #else: 
     #   print 'No matches.'
        
    idx = nltk.text.ContextIndex([word.lower() for word in Corpus])
    simWords = []
    for word in nltk.word_tokenize(keyword):
        simWords.append(idx.similar_words(word))
        if word != None:
            print '\n'.join(idx.similar_words(word))
    return list(itertools.chain.from_iterable(simWords))

In [12]:
# just a toy example for now 
CA_words = ['abstract reasoning','attention','arousal']
def CA_word_presence(Corpus,CA_terms):
    """
    Returns the words from CA that are present in the CA
    
    TO DO: Generalize to entering tasks 
    
    Attributes
    ----------
    
    Corpus : concatenated list of abstracts
    """    
    mask = []
    for ca_idx in xrange(len(CA_terms)):
        if CA_terms[ca_idx] in Corpus:
            mask.append(True)
        else:
            mask.append(False)
            
    PresentTerms = list(compress(CA_words, mask))
    print '\n'.join(word for word in PresentTerms)
    
    return PresentTerms
    

### Scratchpad

In [ ]:
"""
TO DO:

1. Test functions with lisc instead of erpsc
2. Find out the CA terms can be checked within the corpus
3. Submit a pull request to add a version of this notebook to NHW repo
"""

In [13]:
Corpus = get_corpus(dat)

Iterating over 4992 articles for extracting words


In [14]:
get_common_words(Corpus,10)

Term       Frequency  
----       ---------  
p300       9910       
patients   3650       
amplitude  3050       
stimuli    2503       
subjects   2444       
latency    2436       
study      2431       
potentials 2408       
auditory   2286       
task       2268       

 Total Word Count : 442877


In [19]:
sim = get_similarity(Corpus,dat)

cognitive
working
plus
integration
structure
effects
potentials
age
stimuli
abilities
followed
findings
p3b
executive
brain
signs
morphology
curcumin
children
prodromal


In [20]:
sim

[u'cognitive',
 u'working',
 u'plus',
 u'integration',
 u'structure',
 u'effects',
 u'potentials',
 u'age',
 u'stimuli',
 u'abilities',
 u'followed',
 u'findings',
 u'p3b',
 u'executive',
 u'brain',
 u'signs',
 u'morphology',
 u'curcumin',
 u'children',
 u'prodromal']

In [218]:
def test_neighbors(Corpus,keyword)
    
    plain_list = list(Corpus)
    max_instances = 10

    indices = [i for i, x in enumerate(plain_list) if x == "p300"]

    check_first = indices[:max_instances]

    for i in range(max_instances):
        print('Occurence %d of p300 preceded by %s and followed by %s.') \
        %(check_first[i],plain_list[i-1],plain_list[i+1])

Occurence 31 of p300 preceded by activity and followed by rotation.
Occurence 60 of p300 preceded by mental and followed by body.
Occurence 77 of p300 preceded by rotation and followed by parts.
Occurence 91 of p300 preceded by body and followed by involves.
Occurence 174 of p300 preceded by parts and followed by sequential.
Occurence 181 of p300 preceded by involves and followed by cognitive.
Occurence 194 of p300 preceded by sequential and followed by processes.
Occurence 199 of p300 preceded by cognitive and followed by including.
Occurence 303 of p300 preceded by processes and followed by visual.
Occurence 320 of p300 preceded by including and followed by processing.
